In [ ]:
CREATE DATABASE Supplements;

In [4]:
USE Supplements;

Commands completed successfully.

Total execution time: 00:00:00.003

In [ ]:
CREATE TABLE Customers(
    CustomerID int primary key identity(1, 1),
    CustomerName varchar(255) not null,
    CustomerEmail varchar(255) not null,
    CustomerPhone varchar(255) not null 
);

In [6]:
CREATE TABLE Products(
    ProductID int primary key identity(1,1),
    ProductName varchar(255) not null
);

Commands completed successfully.

Total execution time: 00:00:00.021

In [7]:
CREATE TABLE CustomerProducts(
    CustomerID int,
    ProductID int,
    PRIMARY KEY (CustomerID, ProductID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);

Commands completed successfully.

Total execution time: 00:00:00.036

In [10]:
CREATE TYPE dbo.StringList AS TABLE (
    Value varchar(255)
);


Commands completed successfully.

Total execution time: 00:00:00.042

# create a stored procedure that inserts data in tables that are in a m:n relationship; if one insert fails, all the operations performed by the procedure must be rolled back (grade 3);


In [11]:
CREATE PROCEDURE InsertCustomerWithProducts
    @CustomerName varchar(255),
    @CustomerEmail varchar(255),
    @CustomerPhone varchar(255),
    @ProductNames dbo.StringList READONLY -- TVP (Table-Valued Parameter)
AS
BEGIN
    SET NOCOUNT ON;
    BEGIN TRY
        BEGIN TRANSACTION;

        -- Insert into Customers
        INSERT INTO Customers (CustomerName, CustomerEmail, CustomerPhone)
        VALUES (@CustomerName, @CustomerEmail, @CustomerPhone);

        DECLARE @CustomerID int = SCOPE_IDENTITY();

        DECLARE @ProductID int;

        DECLARE cur CURSOR FOR SELECT Value FROM @ProductNames;
        OPEN cur;

        DECLARE @ProdName varchar(255);
        FETCH NEXT FROM cur INTO @ProdName;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            -- Insert product if it doesn't exist, get ID
            IF NOT EXISTS (SELECT 1 FROM Products WHERE ProductName = @ProdName)
            BEGIN
                INSERT INTO Products (ProductName) VALUES (@ProdName);
            END

            SELECT @ProductID = ProductID FROM Products WHERE ProductName = @ProdName;

            -- Insert into junction table
            INSERT INTO CustomerProducts (CustomerID, ProductID)
            VALUES (@CustomerID, @ProductID);

            FETCH NEXT FROM cur INTO @ProdName;
        END
        CLOSE cur;
        DEALLOCATE cur;

        COMMIT;
    END TRY
    BEGIN CATCH
        ROLLBACK;
        THROW;
    END CATCH
END;


Commands completed successfully.

Total execution time: 00:00:00.052

In [12]:
DECLARE @Products dbo.StringList;
INSERT INTO @Products VALUES ('Laptop'), ('Smartphone');

EXEC InsertCustomerWithProducts
    @CustomerName = 'Alice Johnson',
    @CustomerEmail = 'alice@example.com',
    @CustomerPhone = '1234567890',
    @ProductNames = @Products;


(2 rows affected)

Total execution time: 00:00:00.113

In [13]:
SELECT * FROM Customers
SELECT * FROM Products

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.053

CustomerID,CustomerName,CustomerEmail,CustomerPhone
1,Alice Johnson,alice@example.com,1234567890


ProductID,ProductName
1,Laptop
2,Smartphone


# create a stored procedure that inserts data in tables that are in a m:n relationship; if an insert fails, try to recover as much as possible from the entire operation: for example, if the user wants to add a book and its authors, succeeds creating the authors, but fails with the book, the authors should remain in the database (grade 5);

In [14]:
CREATE TABLE Services (
    ServiceID INT PRIMARY KEY IDENTITY(1,1),
    ServiceName VARCHAR(255) NOT NULL
);

CREATE TABLE CustomerServices (
    CustomerID INT,
    ServiceID INT,
    PRIMARY KEY (CustomerID, ServiceID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (ServiceID) REFERENCES Services(ServiceID)
);


Commands completed successfully.

Total execution time: 00:00:00.039

In [6]:
CREATE PROCEDURE InsertCustomerWithServices_Partial
    @CustomerName VARCHAR(255),
    @CustomerEmail VARCHAR(255),
    @CustomerPhone VARCHAR(255),
    @ServiceNames dbo.StringList READONLY
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @CustomerID INT;
    DECLARE @ServiceID INT;
    DECLARE @ServiceName VARCHAR(255);

    -- Try to insert customer first
    BEGIN TRY
        INSERT INTO Customers (CustomerName, CustomerEmail, CustomerPhone)
        VALUES (@CustomerName, @CustomerEmail, @CustomerPhone);

        SET @CustomerID = SCOPE_IDENTITY();
    END TRY
    BEGIN CATCH
        PRINT 'Failed to insert customer.';
        RETURN;
    END CATCH

    -- Process each service individually
    DECLARE service_cursor CURSOR FOR SELECT Value FROM @ServiceNames;
    OPEN service_cursor;
    FETCH NEXT FROM service_cursor INTO @ServiceName;

    WHILE @@FETCH_STATUS = 0
    BEGIN
        BEGIN TRY
            -- Insert service if not exists
            IF NOT EXISTS (SELECT 1 FROM Services WHERE ServiceName = @ServiceName)
            BEGIN
                INSERT INTO Services (ServiceName) VALUES (@ServiceName);
            END

            SELECT @ServiceID = ServiceID FROM Services WHERE ServiceName = @ServiceName;

            -- Link customer to service
            IF NOT EXISTS (
                SELECT 1 FROM CustomerServices WHERE CustomerID = @CustomerID AND ServiceID = @ServiceID
            )
            BEGIN
                INSERT INTO CustomerServices (CustomerID, ServiceID)
                VALUES (@CustomerID, @ServiceID);
            END
        END TRY
        BEGIN CATCH
            PRINT '⚠️ Failed to link or insert service: ' + @ServiceName;
        END CATCH

        FETCH NEXT FROM service_cursor INTO @ServiceName;
    END

    CLOSE service_cursor;
    DEALLOCATE service_cursor;

    PRINT 'Customer inserted. Services linked where possible.';
END;
    

Commands completed successfully.

Total execution time: 00:00:00.026

In [7]:
DECLARE @Services dbo.StringList;
INSERT INTO @Services VALUES ('Internet'), ('Cable TV'), ('Gym Access');

EXEC InsertCustomerWithServices_Partial
    @CustomerName = 'John Doe',
    @CustomerEmail = 'john@example.com',
    @CustomerPhone = '1234567890',
    @ServiceNames = @Services;


(3 rows affected)

Customer inserted. Services linked where possible.

Total execution time: 00:00:00.075

# reproduce the following concurrency issues under pessimistic isolation levels: dirty reads, non-repeatable reads, phantom reads, and a deadlock (4 different scenarios); you can use stored procedures and / or stand-alone queries; find solutions to solve / workaround the concurrency issues (grade 9);

## dirty read (uncommited)

In [8]:
CREATE TABLE Accounts (
    AccountID INT PRIMARY KEY IDENTITY(1,1),
    AccountHolder VARCHAR(100),
    Balance INT
);

INSERT INTO Accounts (AccountHolder, Balance) VALUES ('Alice', 1000), ('Bob', 1500);


(2 rows affected)

Total execution time: 00:00:00.039

In [9]:
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
BEGIN TRAN;
UPDATE Accounts SET Balance = Balance - 100 WHERE AccountHolder = 'Alice';

(1 row affected)

Total execution time: 00:00:00.060

In [10]:
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT * FROM Accounts WHERE AccountHolder = 'Alice';

(1 row affected)

Total execution time: 00:00:00.031

AccountID,AccountHolder,Balance
1,Alice,900


In [11]:
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;
SELECT * FROM Accounts WHERE AccountHolder = 'Alice';


(1 row affected)

Total execution time: 00:00:00.027

AccountID,AccountHolder,Balance
1,Alice,900


In [12]:
-- solution: avoid READ UNCOMMITTED; use READ COMMITTED or higher
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;

Commands completed successfully.

Total execution time: 00:00:00.010

## non-repeatable read (read committed)

In [13]:
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
BEGIN TRAN;
SELECT Balance FROM Accounts WHERE AccountHolder = 'Bob';

(1 row affected)

Total execution time: 00:00:00.025

Balance
1500


In [14]:
BEGIN TRAN;
UPDATE Accounts SET Balance = Balance + 500 WHERE AccountHolder = 'Bob';
COMMIT;

(1 row affected)

Total execution time: 00:00:00.023

In [15]:
SELECT Balance FROM Accounts WHERE AccountHolder = 'Bob';
COMMIT;


(1 row affected)

Total execution time: 00:00:00.019

Balance
2000


In [16]:
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;


Commands completed successfully.

Total execution time: 00:00:00.005

## phantom read(repeatable read)

In [23]:
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
BEGIN TRAN;
SELECT * FROM Accounts WHERE Balance >= 1000;

(3 rows affected)

Total execution time: 00:00:00.006

AccountID,AccountHolder,Balance
1,Alice,1100
2,Bob,1800
3,Charlie,1200


In [24]:
BEGIN TRAN;
INSERT INTO Accounts (AccountHolder, Balance) VALUES ('Charlie', 1200);
COMMIT;

(1 row affected)

Total execution time: 00:00:00.003

In [25]:
SELECT * FROM Accounts WHERE Balance >= 1000;
COMMIT;

(4 rows affected)

Total execution time: 00:00:00.004

AccountID,AccountHolder,Balance
1,Alice,1100
2,Bob,1800
3,Charlie,1200
4,Charlie,1200


In [20]:
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
-- use SERIALIZABLE isolation level to lock ranges

Commands completed successfully.

Total execution time: 00:00:00.013

## deadlock

In [22]:
BEGIN TRAN;
UPDATE Accounts SET Balance = Balance + 100 WHERE AccountHolder = 'Alice';
WAITFOR DELAY '00:00:05';
UPDATE Accounts SET Balance = Balance - 100 WHERE AccountHolder = 'Bob';
-- COMMIT;


(1 row affected)

(1 row affected)

Total execution time: 00:00:05.008

In [ ]:
BEGIN TRAN;
UPDATE Accounts SET Balance = Balance - 200 WHERE AccountHolder = 'Bob';
WAITFOR DELAY '00:00:05';
UPDATE Accounts SET Balance = Balance + 200 WHERE AccountHolder = 'Alice';
-- COMMIT;


In [35]:
CREATE TABLE Books (
    BookID INT PRIMARY KEY IDENTITY(1,1),
    Title VARCHAR(200),
    Stock INT
);

INSERT INTO Books (Title, Stock) VALUES ('The Great Gatsby', 5);
GO


: Msg 3952, Level 16, State 1, Line 7
Snapshot isolation transaction failed accessing database 'Supplements' because snapshot isolation is not allowed in this database. Use ALTER DATABASE to allow snapshot isolation.

Total execution time: 00:00:00.041

In [34]:
SET TRANSACTION ISOLATION LEVEL SNAPSHOT;
BEGIN TRAN;

SELECT Stock FROM Books WHERE BookID = 1;
-- The transaction is open here; do NOT run commit yet.

: Msg 208, Level 16, State 1, Line 4
Invalid object name 'Books'.

Total execution time: 00:00:00.008